## Generic imports

In [12]:
from dotenv import load_dotenv
import logging
import os
import sys

In [13]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [14]:
if not load_dotenv():
    logger.error("No .env file found")

## Setup Haystack pipeline for indexing

### Basic imports for pipeline

In [19]:
from haystack_integrations.document_stores.pinecone import PineconeDocumentStore
from haystack.components.embedders import OpenAIDocumentEmbedder
from haystack.utils import Secret
from haystack.document_stores.types.policy import DuplicatePolicy
from haystack.components.writers import DocumentWriter
import os

### Creating some classes
Haystack does not allow repeated use of the same instance in its pipelines. To combat, I readily create methods here to instantiate all (or some) classes required in my pipeline

In [ ]:

def create_docstore() -> PineconeDocumentStore:
    return PineconeDocumentStore(
        api_key=Secret.from_env_var("PINECONE_API_KEY"),
        index="archiefutrecht", # is nu statisch, raad aan gewoon in .env te zetten
        dimension=1536, # text-embedding-3-small
    )

def create_document_embedder() -> OpenAIDocumentEmbedder:
    return OpenAIDocumentEmbedder(
        model="text-embedding-3-small",
        api_key=Secret.from_env_var("OPENAI_API_KEY"),
        meta_fields_to_embed=[] # Zorgt ervoor dat niet alleen tekst in embedding wordt meegenomen maar ook gespecificeerde metadata. Vet handig voor als je belangrijke metadata genereert.
    )
    
def create_document_writer(docstore) -> DocumentWriter:
    return DocumentWriter(document_store=docstore, policy=DuplicatePolicy.OVERWRITE) 


### Pipeline

In [ ]:
from haystack import Pipeline
from haystack.components.converters import PyPDFToDocument
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter

In [ ]:
def create_indexing_pipeline() -> Pipeline:
    pipeline = Pipeline()
    
    converter = PyPDFToDocument()
    cleaner = DocumentCleaner()
    splitter = DocumentSplitter(split_by="sentence", split_length=5)
    # enricher = DocumentEnricher() Is for later when we have a defined set of metadata I can generate
    embedder = create_document_embedder()
    writer = create_document_writer(create_docstore())
    
    pipeline.add_component("converter", converter)
    pipeline.add_component("cleaner", cleaner)
    pipeline.add_component("splitter", splitter)
    # pipeline.add_component("enricher", enricher)
    pipeline.add_component("embedder", embedder)
    pipeline.add_component("writer", writer)
    
    pipeline.connect("converter", "cleaner")
    pipeline.connect("cleaner", "splitter")
    # pipeline.connect("splitter", "enricher")
    pipeline.connect("splitter", "embedder")
    pipeline.connect("embedder", "writer")
    